In [1]:
!pip install pyLDAvis

In [2]:
import pandas as pd
import numpy as np

# NLTK
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
%matplotlib inline
import seaborn as sns

# Model 1: Data Science Job Post

In [96]:
job_1 = 'Masters Degree in Operations Research, Applied Statistics, Data Mining, Machine Learning or a related quantitative \
discipline.Highly skilled at R Python programming for data science. Deep understanding of statistical and machine-learning \
approaches, including supervised (regression and classification techniques) and unsupervised (clustering) learning, as well as \
recommendation methods. Use of graphical packages, like ggplot2, matplotlib, plotly, or seaborn, to visualize outcomes to a wide \
audience. 4-5+ years’ experience delivering world-class data science outcomes, you solve complex analytical problems using quantitative \
approaches with your unique blend of analytical, mathematical and technical skills.Excellent communication skills, not only with \
other analytically-minded individuals, but a wider audience.Experience with cloud computing environments and \
tooling (e.g. AWS, Azure, Domino Data Lab). Familiarity with the software development lifecycle. Ability to independently navigate \
the data science process, ranging from identifying a business problem, acquiring data, experimenting with solutions, and sharing \
results. Familiarity and background in health care data analytics and use of protected health care data.'

job_2 = 'In pursuit of an undergraduate, graduate, or PhD degree at a four-year institution in Data Science, Math, Physics, Statistics,\
Computer Science,  Engineering, or related field.Deep desire to manipulate and analyze complex, high-volume, highly dimensional\
data from varying sources.Passion for empirical research and answering hard questions with data and models.Knowledge of statistical\
concepts such as correlation, linear regression, optimization, and how they can be applied.Exposure to statistical analysis\
tools/frameworks such as NumPy/SciPy/Pandas/SciKit learn, or SparkML.Expertise in Machine Learning/Cognitive Computing algorithms \
and techniques would be a bonus.Excellent problem solver with the ability to take initiative.Able to present findings and communicate \
effectively.Able to work collaboratively across teams and leverage constructive feedback.Naturally curious and skeptical.'

job_3 = 'Experience in data mining, data analysis, and predictive modelingPrevious work experience in business intelligence and \
data analytics Proven experience utilizing statistical techniques to identify key insights from data to solve business Problems'

job_4 = 'Work with a team of data scientists, machine learning engineers, software engineers and QA engineers.Perform data \
collection, preprocessing, feature engineering, data visualization and analysis.Build automation of data collection and \
preprocessing.Build models to address business problems.Engage with lines of business, users and analysts to explore and \
prototype opportunities and use-cases exploiting data and the application of cognitive and machine learning technologies.\
Design, develop, test and support Cognitive microservices to operationalize and productize deployment of resulting models and \
cognitive solutions.Modern, object-oriented or functional programming experience, (Java, C++, Python, Scala, SQL, R)\
Data Science and Machine Learning Frameworks (R, Apache Spark / MLlib, TensorFlow, Scikit-learn, etc…)Natural Language \
Processing (NLTK, CoreNLP, Gensim, Spacy, etc..)Experience with Big-Data technologies and cloud (AWS, other)Linux / Bash scripting\
Relational databases (Oracle, PostgreSQL, MySQL, etc.)Agile development methodologyCI/CD Development environments \
and tools (GIT, Maven, Jenkins, etc)RESTful Microservice APIsStrong analytical skills. Previous experience or education \
focused on statistics or data science is valuable.Bachelors degree in Computer Science or related degree.At least 1 year of \
experience through internships or field related work experience.Experience in working on an Open Source project a plus but \
not required.Good English language skillsAppreciate the value of diversity (in all its forms) brings to our team and companyPh.D. \
or M.S. in Computer Science, Statistics, or related field.Proficient with at least one major programming language (Java, Scala, \
C++, Python).Experience with supervised and unsupervised machine learning algorithms.Strong grasp of data structures and algorithms.\
Good communication skills with both technical and business people. Ability to speak at a level appropriate for the audience.Cares\
deeply about agile software practices, data driven development, and collaborating with other researchers and engineers.'

job_5 = 'Design and implement data analysis, data mining, research, analysis, modeling strategies and best practices Use analytical\
expertise to spot key business insights, trends, and opportunities in our vast database Help interpret and communicate your findings\
in a way that is easily understood by those less familiar with data Collaborate with a dynamic Product and Development team in delivering\
data products and insights to clients Establish and maintain relationships with distributed cross-functional teams to leverage domain and \
industry expertise Identify missing and inaccurate data in our platforms; seek out supplements and replacements to improve our data sets\
to the greatest extent possible Advise teams and clients as a subject matter expert in data science Bachelor’s degree in a quantitative\
field such as Statistics, Computer Science, Economics, or Mathematics; or equivalent work experience Relevant, successful professional \
industry experience Proficiency with database and analysis tools like R, SQL, Python, and others Expertise in statistical modeling, data\
collection, aggregation, analysis, machine learning, and productization Self-driven passion for finding and collaboratively solving \
problems Strong understanding of the benefits and flaws in the application of various methods'

job_6 = 'Bachelors or graduate degree in statistics/computer science or related field plus a minimum of 3 years applied analytics \
experience End-to-end experience developing approach/methodology given business need, ETL, data quality analysis, model development  \
and validation (machine learning or statistical), scoring, presentation of results and deployment into production environment\
(including AWS, Azure, Google Cloud) Strong experience with predictive models and clustering (segmentation) Strong/Expert SQL, \
Python, R, in a Hadoop environment H2O experience a plus Experience with Tableau, Birst, Microsoft Excel and PowerPoint Travel\
Requirement may be required, up to 20% Strong team player, flexible, and creative Experience with marketing related analytics\
channels/metrics, especially digital Passion/Curiosity around new techniques, ie deep learning Identity resolution experience, \
data with multiple levels and sources – aggregating & connecting Ability to learn and lead with minimal oversight, work on several \
projects at same time Strong communication and presentation skills, ability to maintain timeline'

## Step 1: Preprocess our text.

In [97]:
def text_process(text):
    '''
    Takes in a string of text, then performs the following:
        1. Tokenizes and removes punctuation
        2. Removes stopwords
        3. Stems
        4. Returns a list of the cleaned text
    '''
    if pd.isnull(text):
        return []
    # tokenizing and removing punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    
    # removing any stopwords
    text_processed = [word.lower() for word in text_processed if word.lower() not in stopwords.words('english')]
    
    # steming (commented out to get the non stemmed words)
#     porter_stemmer = PorterStemmer()
    
#     text_processed = [porter_stemmer.stem(word) for word in text_processed]
    
    try:
        text_processed.remove('b')
    except: 
        pass

    return text_processed ## <-- one small tweak

In [99]:
texts = [text_process(job_1),
         text_process(job_2),
         text_process(job_3),
         text_process(job_4),
         text_process(job_5),
        text_process(job_6)]

## Step 2: Fit LDA Model.

In [100]:
from gensim import corpora, models
# from scipy.special.logsumexp import 
import pyLDAvis.gensim

In [101]:
pyLDAvis.enable_notebook()

In [102]:
np.random.seed(42) # want to have reproducable resuslts

In [103]:
dictionary = corpora.Dictionary(texts)

corpus = [dictionary.doc2bow(text) for text in texts] # bow = bag of words


# Instantiate the model
ldamodel = models.ldamodel.LdaModel(corpus=corpus,
                                   id2word=dictionary,
                                    num_topics=5,
                                   passes=5, # similar to epochs in neural networks
                                   minimum_probability=0)

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elog

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elog

## Step 3: Visualize LDA model.

In [104]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      56.276395        1       1  0.095966  0.003464
1      43.201553        1       2 -0.098302  0.003470
2       0.174038        1       3  0.003030  0.005785
3       0.174012        1       4 -0.001649 -0.001276
0       0.174001        1       5  0.000954 -0.011443, topic_info=     Category       Freq          Term      Total  loglift  logprob
term                                                               
6     Default  26.000000          data  26.000000  30.0000  30.0000
18    Default   9.000000       science   9.000000  29.0000  29.0000
48    Default  17.000000    experience  17.000000  28.0000  28.0000
9     Default   9.000000      learning   9.000000  27.0000  27.0000
163   Default   6.000000      analysis   6.000000  26.0000  26.0000
8     Default   7.000000       machine   7.000000  25.0000  25.0000
88    Default   6.000000      business   6.000000  24.0000  24.0000
5     Default   5.000000    statistics   5.000000  23.0000  23.0000
10    Default   5.000000       related   5.000000  22.0000  22.0000
267   Default   5.000000        strong   5.000000  21.0000  21.0000
141   Default   3.000000     cognitive   3.000000  20.0000  20.0000
1     Default   5.000000        degree   5.000000  19.0000  19.0000
21    Default   4.000000   statistical   4.000000  18.0000  18.0000
216   Default   3.000000           etc   3.000000  17.0000  17.0000
174   Default   3.000000     engineers   3.000000  16.0000  16.0000
80    Default   5.000000   development   5.000000  15.0000  15.0000
153   Default   5.000000          work   5.000000  14.0000  14.0000
15    Default   4.000000             r   4.000000  13.0000  13.0000
16    Default   4.000000        python   4.000000  12.0000  12.0000
109   Default   4.000000      computer   4.000000  11.0000  11.0000
33    Default   3.000000           use   3.000000  10.0000  10.0000
82    Default   4.000000       ability   4.000000   9.0000   9.0000
171   Default   2.000000      insights   2.000000   8.0000   8.0000
61    Default   4.000000        skills   4.000000   7.0000   7.0000
7     Default   2.000000        mining   2.000000   6.0000   6.0000
55    Default   3.000000      problems   3.000000   5.0000   5.0000
125   Default   3.000000        models   3.000000   4.0000   4.0000
98    Default   2.000000     analytics   2.000000   3.0000   3.0000
111   Default   3.000000         field   3.000000   2.0000   2.0000
27    Default   3.000000    techniques   3.000000   1.0000   1.0000
...       ...        ...           ...        ...      ...      ...
148    Topic5   0.003229          able   1.892122  -0.0195  -5.9054
89     Topic5   0.003229       problem   1.925972  -0.0371  -5.9053
18     Topic5   0.003766       science   9.943944  -1.5249  -5.7515
62     Topic5   0.003213     excellent   1.930559  -0.0446  -5.9104
25     Topic5   0.003202    regression   1.929177  -0.0472  -5.9137
8      Topic5   0.003524       machine   7.545922  -1.3153  -5.8179
96     Topic5   0.003195        health   1.959154  -0.0648  -5.9158
42     Topic5   0.003190      outcomes   1.951899  -0.0626  -5.9174
9      Topic5   0.003538      learning   9.171827  -1.5065  -5.8139
4      Topic5   0.003257       applied   2.757461  -0.3874  -5.8967
5      Topic5   0.003361    statistics   5.134523  -0.9776  -5.8651
1      Topic5   0.003355        degree   5.140692  -0.9806  -5.8670
54     Topic5   0.003210    analytical   2.748190  -0.3985  -5.9111
48     Topic5   0.003505    experience  17.443091  -2.1586  -5.8232
33     Topic5   0.003232           use   3.558380  -0.6502  -5.9045
44     Topic5   0.003191      audience   2.741785  -0.4021  -5.9170
3      Topic5   0.003186      research   2.736893  -0.4018  -5.9186
10     Topic5   0.003261       related   5.958849  -1.1567  -5.8954
27     Topic5   0.003203    techniques   3.599464  -0.6704  -5.9132
97     Topic5   0.003144          care   1.

## Step 4: Update model with new data!

In [105]:
job_7 = 'PhD degree in computer science, or related technical, math, or scientific field Strong working knowledge of \
deep learning, machine learning and statistics. Hands on experience building models with deep learning frameworks like \
MXNet, Tensorflow, Caffe, Torch, Theano or similar. Experience in using Python, R or Matlab or other statistical/machine \
learning software Strong communication and data presentation skills The motivation to achieve results in a fast-paced \
environment. Experience with statistical modeling / machine learning Strong attention to detail Comfortable working in a \
fast paced, highly collaborative, dynamic work environment Ability to think creatively and solve problems'

In [106]:
new_doc = text_process(job_7)

In [107]:
ldamodel.update([dictionary.doc2bow(new_doc)])

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elog

In [108]:
job_8 = 'ability to tell meaningful stories with data. Programming skills in Python Experience with SQL, database modeling and \
data warehousing principles. Creativity to go beyond current tools to deliver best solution to the problem. Data visualization \
experience in Matplotlib, Tableau, ggplot2 or a similar tool. Experience with data science algorithms including decision trees, \
probability networks, association rules, clustering, regression, and neural networks. Excellent interpersonal, written, and verbal \
communication skills. Ability and comfort working independently and making key decisions on projects. Build and monitor effective \
reporting and alerting for fraud and operational health of a line of business.  Perform ad-hoc and re-occurring statistical and data \
science analyses.  Work closely with data warehouse architects and software developers to generate seamless business intelligence \
solutions for end users.  Support production analytic solutions.  Present results of analyses to business units.'

In [109]:
new_doc2 = text_process(job_8)

In [110]:
ldamodel.update([dictionary.doc2bow(new_doc2)])

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elog

In [111]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      40.340475        1       1  0.012657  0.055000
4      35.520141        1       2  0.208350 -0.016835
2      12.126072        1       3 -0.073559 -0.012807
3       7.354196        1       4 -0.073695 -0.012665
0       4.659117        1       5 -0.073752 -0.012693, topic_info=     Category       Freq             Term      Total  loglift  logprob
term                                                                  
6     Default  27.000000             data  27.000000  30.0000  30.0000
250   Default  16.000000          working  16.000000  29.0000  29.0000
142   Default  12.000000       algorithms  12.000000  28.0000  28.0000
48    Default  15.000000       experience  15.000000  27.0000  27.0000
88    Default  13.000000         business  13.000000  26.0000  26.0000
181   Default  11.000000            build  11.000000  25.0000  25.0000
61    Default   9.000000           skills   9.000000  24.0000  24.0000
82    Default   9.000000          ability   9.000000  23.0000  23.0000
280   Default   5.000000         modeling   5.000000  22.0000  22.0000
94    Default   5.000000          results   5.000000  21.0000  21.0000
21    Default   5.000000      statistical   5.000000  20.0000  20.0000
18    Default  10.000000          science  10.000000  19.0000  19.0000
170   Default   4.000000              key   4.000000  18.0000  18.0000
287   Default   4.000000         database   4.000000  17.0000  17.0000
29    Default   4.000000       clustering   4.000000  16.0000  16.0000
23    Default   4.000000        including   4.000000  15.0000  15.0000
96    Default   4.000000           health   4.000000  14.0000  14.0000
345   Default   4.000000              end   4.000000  13.0000  13.0000
166   Default   4.000000     intelligence   4.000000  12.0000  12.0000
282   Default   4.000000             best   4.000000  11.0000  11.0000
363   Default   4.000000          tableau   4.000000  10.0000  10.0000
357   Default   4.000000       production   4.000000   9.0000   9.0000
393   Default   4.000000         projects   4.000000   8.0000   8.0000
37    Default   4.000000          ggplot2   4.000000   7.0000   7.0000
38    Default   4.000000       matplotlib   4.000000   6.0000   6.0000
83    Default   4.000000    independently   4.000000   5.0000   5.0000
92    Default  10.000000        solutions  10.000000   4.0000   4.0000
63    Default   5.000000    communication   5.000000   3.0000   3.0000
211   Default   4.000000              sql   4.000000   2.0000   2.0000
133   Default   2.000000       frameworks   2.000000   1.0000   1.0000
...       ...        ...              ...        ...      ...      ...
20     Topic5   0.080163    understanding   0.914473   0.6321  -5.9809
145    Topic5   0.080185           solver   0.929210   0.6163  -5.9806
156    Topic5   0.080219            teams   0.954065   0.5904  -5.9802
154    Topic5   0.080200  collaboratively   0.941677   0.6032  -5.9804
134    Topic5   0.080185            numpy   0.929303   0.6162  -5.9806
124    Topic5   0.080179        questions   0.929324   0.6162  -5.9807
151    Topic5   0.080189      communicate   0.941680   0.6031  -5.9806
146    Topic5   0.080178             take   0.929252   0.6162  -5.9807
137    Topic5   0.080211           scikit   0.969090   0.5746  -5.9803
162    Topic5   0.080177        skeptical   0.929316   0.6161  -5.9807
100    Topic5   0.080174          pursuit   0.929260   0.6162  -5.9808
6      Topic5   0.081116             data  27.382573  -2.7554  -5.9691
18     Topic5   0.080603          science  10.813596  -1.8327  -5.9754
9      Topic5   0.080461         learning   5.301493  -1.1216  -5.9772
8      Topic5   0.080452          machine   3.935979  -0.8239  -5.9773
48     Topic5   0.080441       experience  15.705841  -2.2079  -5.9774
5      Topic5   0.080351       statistics   1.931143  -0.1131  -5.9786
1      Topic5   0.080347     

In [114]:
# TOPIC 1
ldamodel.print_topics(num_words=10)[1]

(1,
 '0.059*"working" + 0.044*"algorithms" + 0.037*"build" + 0.024*"present" + 0.023*"support" + 0.023*"users" + 0.023*"visualization" + 0.023*"perform" + 0.020*"data" + 0.012*"programming"')

In [116]:
# TOPIC 2
ldamodel.print_topics(num_words=10)[4]

(4,
 '0.089*"data" + 0.050*"experience" + 0.044*"business" + 0.032*"skills" + 0.032*"ability" + 0.030*"science" + 0.028*"solutions" + 0.020*"statistical" + 0.018*"modeling" + 0.018*"results"')